In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# for dirname, _, filenames in os.walk("./"):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import pandas as pd
import torch
import torch.nn as nn
from scipy import stats

# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project

Given 30 distinct datasets, each consisting of 26 color images of a lowercase alphabetical letter in cursive handwriting, train a neural network that can read in similar images of cursive handwriting and identify the letter that is written with a moderately high degree of accuracy (ideally ~80s/90s)

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

I downloaded all files from the Google Drive directory, then did some manual inspection with each folder. Based on an initial surface-level analysis, I found that:
- within the S10 directory were 4 directories titled S11, S12, S13, S14.
    - 3 of these 4 nested directories contained the correct amount of lowercase letter images needed
    - S13 contained duplicates for most of the 26 letters
- the S11, S12, S13, and S14 directories not nested inside of the S10 directory were either all incomplete or empty
- there were 2 folders that were duplicates of other existing folders (same images, same names, etc)
- S33 was missing the image for a lowercase j
- S34 was largely incomplete and contained 10 letters
- S35 was completely empty
- many directories either had inconsistent naming (.png for some files and no extension for others), unlabeled data (images were not labeled with the intended letter), and the file extension varied across each directory (.heic, .jpg, .png, etc)

my final decisions for the data batch were to:
- use .png as a universal file extension across all datasets
- rename unlabeled images with their corresponding lowercase letter
    - some datasets were messier than others; hence, some guesswork had to be done for certain directories
- deleted duplicate images in the S13 directory nested within the S10 directory
- use the S11, S12, S13, and S14 directories nested within the S10 directory, and deleted the empty/incomplete directories with the same names outside of S10
- deleted S33, S34, and S35
- renumbered directories so that they would not have gaps

and in the end, our final data consists of:
- 30 distinct directories labeled 1-30 in ./og_data
- 26 color-images of lowercase letters in each directory, labeled (letter).png 

# 3. Explore the Data

Gain insights into the data you have from step 2, making sure to identify any bias

In [16]:
# make sure each directory has 26 images

for n in range(1, 31):
    print(f'\n=======DATASET {n}=======\n')
    path = f"./og_data/{n}"
    filenames = [f for f in os.listdir(path) 
                 if os.path.isfile(os.path.join(path, f)) 
                 and 'checkpoint' not in f]
    
    print(f'length of dataset {n}: {len(filenames)}')
    if len(filenames) != 26: 
        print(sorted(filenames))


=======DATASET 1=======

length of dataset 1: 26

=======DATASET 2=======

length of dataset 2: 26

=======DATASET 3=======

length of dataset 3: 26

=======DATASET 4=======

length of dataset 4: 26

=======DATASET 5=======

length of dataset 5: 26

=======DATASET 6=======

length of dataset 6: 26

=======DATASET 7=======

length of dataset 7: 26

=======DATASET 8=======

length of dataset 8: 26

=======DATASET 9=======

length of dataset 9: 26

=======DATASET 10=======

length of dataset 10: 26

=======DATASET 11=======

length of dataset 11: 26

=======DATASET 12=======

length of dataset 12: 26

=======DATASET 13=======

length of dataset 13: 26

=======DATASET 14=======

length of dataset 14: 26

=======DATASET 15=======

length of dataset 15: 26

=======DATASET 16=======

length of dataset 16: 26

=======DATASET 17=======

length of dataset 17: 25
['a.png', 'b.png', 'c.png', 'd.png', 'e.png', 'f.png', 'g.png', 'h.png', 'i.png', 'j.png', 'k.png', 'l.png', 'm.png', 'o.png', 'p.png'

# 4. Prepare the Data


Apply any data transformations and explain what and why

# 5. Model the data 
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 


# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken

# 8. Launch the Model System
Define your production run code, This should be self sufficient and require only your model pramaters 


In [ ]:
def inference():
    